# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 0: Preparation and import data from s3
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
#Before we continue, we need to install related python package.
import sys

!{sys.executable} -m pip install boto3
!{sys.executable} -m pip install s3fs
!{sys.executable} -m pip install pyspark
!{sys.executable} -m pip install cqlsh

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Do all imports and installs here
import configparser
import pandas as pd
import os
import boto3
import uuid

In [3]:
config = configparser.ConfigParser()
config.read('iam.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config['AWS_CREDS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS_CREDS']['AWS_SECRET_ACCESS_KEY']

client=boto3.client('s3')

global_land_temperature_url = 's3://srk-data-eng-capstone/GlobalLandTemperaturesByCity.csv'
airport_codes_url = 's3://srk-data-eng-capstone/airport-codes_csv.csv'
us_city_demographics_url = 's3://srk-data-eng-capstone/us-cities-demographics.csv'


### Scope the Project and Gather Data

#### Project description:

This project will be separate to multiple parts, and all four dataset will be used. 

Before we talk about the details, we need to know the characteristics of relational DB and non-relational DB.

For relational DB, its characteristics is low redundancy and high completeness, which means it is very suitable for small or medium size data, and the database does not change so much. In our case, we should store temperature, airport code and US cities demographic data into a relational database that meets 3NF because it does not always change so much and the volume of data is not that large.

For non-relational DB, its characteristics is higher elasticity, faster read & write speed and evoving data volume. In our case, we should save I94 data into non-relational DB. Because this piece of data need to make ETL process almost every minutes in real world background, and it need dynamic write and read for real-time data monitoring.


The final solution will work as a a database management system. When user input the time or time period and the column they interested in (e.g., visa type), the system will return related data as a dataframe. For example, a user needs to know where are the busiest airport for investment visa holder(E-1 visa) in 2016 and its basic information such as temperature, and the status of the city such as population(age, majority race, etc.), or when is the peak-time for international student come to the United States and where are they come from.

* Data will be imported from Amazon S3
* Relational DB will be implement on AWS Redshift
* Non-Relationalship DB will be implement on Amazon Keyspace, and data backup will be stored at S3 as parquet format.
* Data cleaning and ETL process will be implement on Amazon EMR with Spark

#### The dataset is going to use in this project are:

* I94 Immigration Data: This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. https://travel.trade.gov/research/reports/i94/historical/2016.html is where the data comes from. There's a sample file so you can take a look at the data in csv format before reading it all in. You do not have to use the entire dataset, just use what you need to accomplish the goal you set at the beginning of the project.
* World Temperature Data: This dataset came from Kaggle. You can read more about it here: https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data.
* U.S. City Demographic Data: This data comes from OpenSoft. You can read more about it here: https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/.
* Airport Code Table: This is a simple table of airport codes and corresponding cities. It comes from here:https://datahub.io/core/airport-codes#data.

In [17]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

while True:
    if os.path.isdir('16_jan.parquet'):
        break
    else:
        sql = '''SELECT id_, cicid, i94yr, i94mon, i94cit, i94res, i94port,
                 arrdate, i94mode, i94addr, depdate, i94bir, i94visa,
                 count, dtadfile, visapost, occup, entdepa, entdepd,
                 entdepu, matflag, biryear, dtaddto, gender, insnum,
                 airline, admnum, fltno, visatype FROM i94'''
        
        i94_url='s3://srk-data-eng-capstone/i94/i94_jan16_sub.sas7bdat'
        i94 = pd.read_sas(i94_url, 'sas7bdat',encoding="ISO-8859-1").drop_duplicates()
        i94['id_'] = pd.Series([uuid.uuid1() for each in range(len(i94))])
        i94['dtadfile'].fillna(-1,inplace=True)
        for each in i94:
            dt=i94[each].dtype
            if dt == 'object':
                i94[each].fillna('Null',inplace=True)
            else:
                i94[each].fillna(-1,inplace=True)
        i94.to_csv('16_jan.csv', index=False)
        df_spark=spark.read.option('header','true').csv('16_jan.csv')
        df_spark.createOrReplaceTempView('i94')
        parquet_data=spark.sql(sql)
        parquet_data.write.parquet('16_jan.parquet',mode='overwrite')
        os.system('aws s3 cp 16_jan.parquet s3://i94-backup --recursive')
        break        



In [26]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1, CERT_REQUIRED
from cassandra.auth import PlainTextAuthProvider
from cassandra import ConsistencyLevel

ssl_context = SSLContext(PROTOCOL_TLSv1)
ssl_context.load_verify_locations('AmazonRootCA1.pem')
ssl_context.verify_mode = CERT_REQUIRED
auth_provider = PlainTextAuthProvider(username=str(config['APACHE_CASSANDRA_CREDS']['CASSANDRA_USERNAME']), password=str(config['APACHE_CASSANDRA_CREDS']['CASSANDRA_PASSWORD']))
cluster = Cluster(['cassandra.eu-west-1.amazonaws.com'], ssl_context=ssl_context, auth_provider=auth_provider, port=9142)

In [28]:
session = cluster.connect()
create_keyspace="""CREATE KEYSPACE IF NOT EXISTS "i94"
                   WITH REPLICATION={'class':'SingleRegionStrategy'}"""
session.execute(create_keyspace)

create_table="""CREATE TABLE IF NOT EXISTS "i94".i94 (
                                                      cicid DOUBLE,
                                                      i94yr DOUBLE,
                                                      i94mon DOUBLE,
                                                      i94cit DOUBLE,
                                                      i94res DOUBLE,
                                                      i94port TEXT,
                                                      arrdate DOUBLE,
                                                      i94mode DOUBLE,
                                                      i94addr TEXT,
                                                      depdate DOUBLE,
                                                      i94bir DOUBLE,
                                                      i94visa DOUBLE,
                                                      count DOUBLE,
                                                      dtadfile DOUBLE,
                                                      visapost TEXT,
                                                      occup TEXT,
                                                      entdepa TEXT,
                                                      entdepd TEXT,
                                                      entdepu TEXT,
                                                      matflag TEXT,
                                                      biryear DOUBLE,
                                                      dtaddto TEXT,
                                                      gender TEXT,
                                                      insnum TEXT,
                                                      airline TEXT,
                                                      admnum DOUBLE,
                                                      fltno TEXT,
                                                      visatype TEXT,
                                                      id_ TEXT,
                                                      PRIMARY KEY(id_)
                ) """
session.execute(create_table)

In [ ]:
origin_sql="""INSERT INTO "i94".i94 ("cicid","i94yr","i94mon","i94cit","i94res","i94port","arrdate","i94mode","i94addr","depdate",
                              "i94bir","i94visa","count","dtadfile","visapost","occup","entdepa","entdepd","entdepu","matflag",
                              "biryear","dtaddto","gender","insnum","airline","admnum","fltno","visatype","id_")
                              VALUES ({0},{1},{2},{3},{4},'{5}',{6},{7},'{8}',{9},
                                      {10},{11},{12},{13},'{14}','{15}','{16}','{17}','{18}','{19}',
                                      {20},'{21}','{22}','{23}','{24}',{25},'{26}','{27}','{28}')"""


total_length=len(pd.read_csv('16_jan.csv'))

with open('{}.csv'.format('16_jan'),'r') as csv:
    #Ignore the first line, which is column head.
    csv=iter(csv)
    next(csv)
    counts=0
    for each in csv:
        lists=[]
        columns=each.split(',')
        cicid=columns[0]
        i94yr=columns[1]
        i94mon=columns[2]
        i94cit=columns[3]
        i94res=columns[4]
        i94port=columns[5]
        arrdate=columns[6]
        i94mode=columns[7]
        i94addr=columns[8]
        depdate=columns[9]
        i94bir=columns[10]
        i94visa=columns[11]
        count=columns[12]
        dtadfile=columns[13]
        visapost=columns[14]
        occup=columns[15]
        entdepa=columns[16]
        entdepd=columns[17]
        entdepu=columns[18]
        matflag=columns[19]
        biryear=columns[20]
        dtaddto=columns[21]
        gender=columns[22]
        insnum=columns[23]
        airline=columns[24]
        admnum=columns[25]
        fltno=columns[26]
        visatype=columns[27]
        id_=columns[28]        
        
        original_list=[cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,
                      count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,
                      airline,admnum,fltno,visatype,id_]
        
        for each in original_list:
            try:
                each = float(each)
                lists.append(each)
            except:
                if each == '':
                    each = None
                    lists.append(each)
                else:
                    lists.append(each)
                    continue                            
        
        formated_sql=origin_sql.format(lists[0],lists[1],lists[2],lists[3],lists[4],lists[5],lists[6],lists[7],lists[8],lists[9],
                      lists[10],lists[11],lists[12],lists[13],lists[14],lists[15],lists[16],lists[17],lists[18],lists[19],
                      lists[20],lists[21],lists[22],lists[23],lists[24],lists[25],lists[26],lists[27],lists[28])
        
        sql=session.prepare(formated_sql)
        sql.consistency_level = ConsistencyLevel.LOCAL_QUORUM        
        session.execute(sql)
        
        print('Import row{} complete.'.format(counts) + '{} remaining.'.format(total_length-counts))
        counts+=1
    


Import row0 complete.2847924 remaining.
Import row1 complete.2847923 remaining.
Import row2 complete.2847922 remaining.
Import row3 complete.2847921 remaining.
Import row4 complete.2847920 remaining.
Import row5 complete.2847919 remaining.
Import row6 complete.2847918 remaining.
Import row7 complete.2847917 remaining.
Import row8 complete.2847916 remaining.
Import row9 complete.2847915 remaining.
Import row10 complete.2847914 remaining.
Import row11 complete.2847913 remaining.
Import row12 complete.2847912 remaining.
Import row13 complete.2847911 remaining.
Import row14 complete.2847910 remaining.
Import row15 complete.2847909 remaining.
Import row16 complete.2847908 remaining.
Import row17 complete.2847907 remaining.
Import row18 complete.2847906 remaining.
Import row19 complete.2847905 remaining.
Import row20 complete.2847904 remaining.
Import row21 complete.2847903 remaining.
Import row22 complete.2847902 remaining.
Import row23 complete.2847901 remaining.
Import row24 complete.2847

Import row200 complete.2847724 remaining.
Import row201 complete.2847723 remaining.
Import row202 complete.2847722 remaining.
Import row203 complete.2847721 remaining.
Import row204 complete.2847720 remaining.
Import row205 complete.2847719 remaining.
Import row206 complete.2847718 remaining.
Import row207 complete.2847717 remaining.
Import row208 complete.2847716 remaining.
Import row209 complete.2847715 remaining.
Import row210 complete.2847714 remaining.
Import row211 complete.2847713 remaining.
Import row212 complete.2847712 remaining.
Import row213 complete.2847711 remaining.
Import row214 complete.2847710 remaining.
Import row215 complete.2847709 remaining.
Import row216 complete.2847708 remaining.
Import row217 complete.2847707 remaining.
Import row218 complete.2847706 remaining.
Import row219 complete.2847705 remaining.
Import row220 complete.2847704 remaining.
Import row221 complete.2847703 remaining.
Import row222 complete.2847702 remaining.
Import row223 complete.2847701 rem

Import row397 complete.2847527 remaining.
Import row398 complete.2847526 remaining.
Import row399 complete.2847525 remaining.
Import row400 complete.2847524 remaining.
Import row401 complete.2847523 remaining.
Import row402 complete.2847522 remaining.
Import row403 complete.2847521 remaining.
Import row404 complete.2847520 remaining.
Import row405 complete.2847519 remaining.
Import row406 complete.2847518 remaining.
Import row407 complete.2847517 remaining.
Import row408 complete.2847516 remaining.
Import row409 complete.2847515 remaining.
Import row410 complete.2847514 remaining.
Import row411 complete.2847513 remaining.
Import row412 complete.2847512 remaining.
Import row413 complete.2847511 remaining.
Import row414 complete.2847510 remaining.
Import row415 complete.2847509 remaining.
Import row416 complete.2847508 remaining.
Import row417 complete.2847507 remaining.
Import row418 complete.2847506 remaining.
Import row419 complete.2847505 remaining.
Import row420 complete.2847504 rem

Import row596 complete.2847328 remaining.
Import row597 complete.2847327 remaining.
Import row598 complete.2847326 remaining.
Import row599 complete.2847325 remaining.
Import row600 complete.2847324 remaining.
Import row601 complete.2847323 remaining.
Import row602 complete.2847322 remaining.
Import row603 complete.2847321 remaining.
Import row604 complete.2847320 remaining.
Import row605 complete.2847319 remaining.
Import row606 complete.2847318 remaining.
Import row607 complete.2847317 remaining.
Import row608 complete.2847316 remaining.
Import row609 complete.2847315 remaining.
Import row610 complete.2847314 remaining.
Import row611 complete.2847313 remaining.
Import row612 complete.2847312 remaining.
Import row613 complete.2847311 remaining.
Import row614 complete.2847310 remaining.
Import row615 complete.2847309 remaining.
Import row616 complete.2847308 remaining.
Import row617 complete.2847307 remaining.
Import row618 complete.2847306 remaining.
Import row619 complete.2847305 rem

Import row795 complete.2847129 remaining.
Import row796 complete.2847128 remaining.
Import row797 complete.2847127 remaining.
Import row798 complete.2847126 remaining.
Import row799 complete.2847125 remaining.
Import row800 complete.2847124 remaining.
Import row801 complete.2847123 remaining.
Import row802 complete.2847122 remaining.
Import row803 complete.2847121 remaining.
Import row804 complete.2847120 remaining.
Import row805 complete.2847119 remaining.
Import row806 complete.2847118 remaining.
Import row807 complete.2847117 remaining.
Import row808 complete.2847116 remaining.
Import row809 complete.2847115 remaining.
Import row810 complete.2847114 remaining.
Import row811 complete.2847113 remaining.
Import row812 complete.2847112 remaining.
Import row813 complete.2847111 remaining.
Import row814 complete.2847110 remaining.
Import row815 complete.2847109 remaining.
Import row816 complete.2847108 remaining.
Import row817 complete.2847107 remaining.
Import row818 complete.2847106 rem

...

Import row23024 complete.2824900 remaining.
Import row23025 complete.2824899 remaining.
Import row23026 complete.2824898 remaining.
Import row23027 complete.2824897 remaining.
Import row23028 complete.2824896 remaining.
Import row23029 complete.2824895 remaining.
Import row23030 complete.2824894 remaining.
Import row23031 complete.2824893 remaining.
Import row23032 complete.2824892 remaining.
Import row23033 complete.2824891 remaining.
Import row23034 complete.2824890 remaining.
Import row23035 complete.2824889 remaining.
Import row23036 complete.2824888 remaining.
Import row23037 complete.2824887 remaining.
Import row23038 complete.2824886 remaining.
Import row23039 complete.2824885 remaining.
Import row23040 complete.2824884 remaining.
Import row23041 complete.2824883 remaining.
Import row23042 complete.2824882 remaining.
Import row23043 complete.2824881 remaining.
Import row23044 complete.2824880 remaining.
Import row23045 complete.2824879 remaining.
Import row23046 complete.2824878

Import row23214 complete.2824710 remaining.
Import row23215 complete.2824709 remaining.
Import row23216 complete.2824708 remaining.
Import row23217 complete.2824707 remaining.
Import row23218 complete.2824706 remaining.
Import row23219 complete.2824705 remaining.
Import row23220 complete.2824704 remaining.
Import row23221 complete.2824703 remaining.
Import row23222 complete.2824702 remaining.
Import row23223 complete.2824701 remaining.
Import row23224 complete.2824700 remaining.
Import row23225 complete.2824699 remaining.
Import row23226 complete.2824698 remaining.
Import row23227 complete.2824697 remaining.
Import row23228 complete.2824696 remaining.
Import row23229 complete.2824695 remaining.
Import row23230 complete.2824694 remaining.
Import row23231 complete.2824693 remaining.
Import row23232 complete.2824692 remaining.
Import row23233 complete.2824691 remaining.
Import row23234 complete.2824690 remaining.
Import row23235 complete.2824689 remaining.
Import row23236 complete.2824688

Import row23401 complete.2824523 remaining.
Import row23402 complete.2824522 remaining.
Import row23403 complete.2824521 remaining.
Import row23404 complete.2824520 remaining.
Import row23405 complete.2824519 remaining.
Import row23406 complete.2824518 remaining.
Import row23407 complete.2824517 remaining.
Import row23408 complete.2824516 remaining.
Import row23409 complete.2824515 remaining.
Import row23410 complete.2824514 remaining.
Import row23411 complete.2824513 remaining.
Import row23412 complete.2824512 remaining.
Import row23413 complete.2824511 remaining.
Import row23414 complete.2824510 remaining.
Import row23415 complete.2824509 remaining.
Import row23416 complete.2824508 remaining.
Import row23417 complete.2824507 remaining.
Import row23418 complete.2824506 remaining.
Import row23419 complete.2824505 remaining.
Import row23420 complete.2824504 remaining.
Import row23421 complete.2824503 remaining.
Import row23422 complete.2824502 remaining.
Import row23423 complete.2824501

Import row23591 complete.2824333 remaining.
Import row23592 complete.2824332 remaining.
Import row23593 complete.2824331 remaining.
Import row23594 complete.2824330 remaining.
Import row23595 complete.2824329 remaining.
Import row23596 complete.2824328 remaining.
Import row23597 complete.2824327 remaining.
Import row23598 complete.2824326 remaining.
Import row23599 complete.2824325 remaining.
Import row23600 complete.2824324 remaining.
Import row23601 complete.2824323 remaining.
Import row23602 complete.2824322 remaining.
Import row23603 complete.2824321 remaining.
Import row23604 complete.2824320 remaining.
Import row23605 complete.2824319 remaining.
Import row23606 complete.2824318 remaining.
Import row23607 complete.2824317 remaining.
Import row23608 complete.2824316 remaining.
Import row23609 complete.2824315 remaining.
Import row23610 complete.2824314 remaining.
Import row23611 complete.2824313 remaining.
Import row23612 complete.2824312 remaining.
Import row23613 complete.2824311

Import row23780 complete.2824144 remaining.
Import row23781 complete.2824143 remaining.
Import row23782 complete.2824142 remaining.
Import row23783 complete.2824141 remaining.
Import row23784 complete.2824140 remaining.
Import row23785 complete.2824139 remaining.
Import row23786 complete.2824138 remaining.
Import row23787 complete.2824137 remaining.
Import row23788 complete.2824136 remaining.
Import row23789 complete.2824135 remaining.
Import row23790 complete.2824134 remaining.
Import row23791 complete.2824133 remaining.
Import row23792 complete.2824132 remaining.
Import row23793 complete.2824131 remaining.
Import row23794 complete.2824130 remaining.
Import row23795 complete.2824129 remaining.
Import row23796 complete.2824128 remaining.
Import row23797 complete.2824127 remaining.
Import row23798 complete.2824126 remaining.
Import row23799 complete.2824125 remaining.
Import row23800 complete.2824124 remaining.
Import row23801 complete.2824123 remaining.
Import row23802 complete.2824122

In [ ]:
while True:
    try:
        os.remove('16_jan.csv')
    except:
        break

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.